# Pandas и большие файлы

In [2]:
import pandas as pd

### Упражнение
Для каждого пользователя user_id из файла sales_db.csv посчитайте самую дорогую покупку (в столбце cost)

In [3]:
df_sales = pd.read_csv('sales_db.csv')
df_sales.head()

,user_id,category,cost
0,1,Электроника,87
1,1,Дача,10
2,1,Дети,2
3,1,Дача,199
4,2,Авто,3


In [13]:
df_sales.groupby(['user_id']).max().head()

,category,cost
user_id,,
1,Электроника,199
2,Авто,35
3,Дети,8
9,Дети,17


In [15]:
df_sales.groupby(['user_id']).max().sort_values(by='cost', ascending = False).head(1)

,category,cost
user_id,,
1,Электроника,199


In [14]:
df_sales.groupby(df_sales.user_id).max()[['cost']]

,cost
user_id,
1,199
2,35
3,8
9,17


In [16]:
df_sales.groupby(df_sales.user_id).agg({'cost' : 'max'})

,cost
user_id,
1,199
2,35
3,8
9,17


In [22]:
previous_user = None
user_lines = []

with open('sales_db.csv', encoding = 'utf8') as f:
    f.readline()
    for line in f:
        line = line.strip().split(',')
        
        user_id, category, cost = line
        print(user_id, category, cost)
        
        if previous_user != None:
            if previous_user != user_id:
                print(max([x[2] for x in user_lines]))
                previous_user = user_id
                user_lines = [line]
            else:
                user_lines.append(line)   
        else:
            previous_user = user_id
            user_lines.append(line)
        

1 Электроника 87
1 Дача 10
1 Дети 2
1 Дача 199
2 Авто 3
87
2 Авто 35
3 Дети 8
35
9 Дети 17
8


# Объединение датафреймов

### Данные со слайдов

In [23]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [24]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [25]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [26]:
visits.groupby('user_id').count().reset_index()

,user_id,source
0,11,2
1,22,1
2,55,1
3,77,1


In [27]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [28]:
purchases_pivot.reset_index()

category,user_id,Авто,Дача,Спорт
0,11,0,0,2
1,22,1,0,0
2,55,0,1,0
3,99,1,0,0


In [29]:
visits_grouped.join(purchases_pivot)

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [30]:
visits_grouped.join(purchases_pivot, how='left')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [31]:
visits_grouped.join(purchases_pivot, how='right')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0,0,2
22,1.0,1,0,0
55,1.0,0,1,0
99,NaN,1,0,0


### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [20]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [33]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

,id,name
0,43018,Марков Илья
1,48329,Зарицкая Елизавета
2,51043,Благова Дарья
3,74943,Слепова Елена
4,75029,Гордецкий Максим


In [34]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

,id,debit
0,51043,34500
1,48329,12400
2,74943,89044
3,75029,5355
4,43018,19800


In [35]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

,id,credit
0,51043,22990
1,48329,2500
2,74943,69880
3,75029,6000
4,43018,29000


In [39]:
clients_debt = clients.merge(earnings, left_on='id', right_on='id')
clients_debt

,id,name,debit
0,43018,Марков Илья,19800
1,48329,Зарицкая Елизавета,12400
2,51043,Благова Дарья,34500
3,74943,Слепова Елена,89044
4,75029,Гордецкий Максим,5355


In [40]:
clients_debt_cred = clients_debt.merge(spending, left_on='id', right_on='id')
clients_debt_cred

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
1,48329,Зарицкая Елизавета,12400,2500
2,51043,Благова Дарья,34500,22990
3,74943,Слепова Елена,89044,69880
4,75029,Гордецкий Максим,5355,6000


In [43]:
clients_debt_cred['buldo'] = clients_debt_cred['debit'] - clients_debt_cred['credit']
clients_debt_cred.sort_values('buldo')

,id,name,debit,credit,buldo
0,43018,Марков Илья,19800,29000,-9200
4,75029,Гордецкий Максим,5355,6000,-645
1,48329,Зарицкая Елизавета,12400,2500,9900
2,51043,Благова Дарья,34500,22990,11510
3,74943,Слепова Елена,89044,69880,19164


In [44]:
len(clients) == len(clients_debt_cred)

True

In [47]:
sum(earnings.debit) == sum(clients_debt_cred.debit)

True

### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [49]:
visits_grouped.join(purchases_pivot, how='inner')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0,0,2
22,1,1,0,0
55,1,0,1,0


### Outer join
Оставляет все строчки

In [50]:
visits_grouped.join(purchases_pivot, how='outer')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0.0,0.0,2.0
22,1.0,1.0,0.0,0.0
55,1.0,0.0,1.0,0.0
77,1.0,NaN,NaN,NaN
99,NaN,1.0,0.0,0.0


# Конкатенация таблиц

In [48]:
a = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_a': [1, 2, 3]})
b = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_b': [3, 4, 5]})

In [51]:
a

,date,value_a
0,2020-01-01,1
1,2020-01-02,2
2,2020-01-03,3


In [52]:
b

,date,value_b
0,2020-01-01,3
1,2020-01-02,4
2,2020-01-03,5


In [53]:
pd.concat([a, b])

C:\Users\AAZam\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,date,value_a,value_b
0,2020-01-01,1.0,NaN
1,2020-01-02,2.0,NaN
2,2020-01-03,3.0,NaN
0,2020-01-01,NaN,3.0
1,2020-01-02,NaN,4.0
2,2020-01-03,NaN,5.0


In [63]:
pd.concat([a, b], sort = True)

,date,value_a,value_b
0,2020-01-01,1.0,NaN
1,2020-01-02,2.0,NaN
2,2020-01-03,3.0,NaN
0,2020-01-01,NaN,3.0
1,2020-01-02,NaN,4.0
2,2020-01-03,NaN,5.0


In [58]:
pd.concat([a, b], sort = True).reset_index()

,index,date,value_a,value_b
0,0,2020-01-01,1.0,NaN
1,1,2020-01-02,2.0,NaN
2,2,2020-01-03,3.0,NaN
3,0,2020-01-01,NaN,3.0
4,1,2020-01-02,NaN,4.0
5,2,2020-01-03,NaN,5.0


In [64]:
pd.concat([a, b], sort = True).reset_index(drop)

NameError: name 'drop' is not defined

In [54]:
# объединение по горизонтали
pd.concat([a, b], axis=1)

,date,value_a,date,value_b
0,2020-01-01,1,2020-01-01,3
1,2020-01-02,2,2020-01-02,4
2,2020-01-03,3,2020-01-03,5


### Дубликаты при объединении таблиц

In [55]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [56]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [65]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [66]:
movies.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True) # можно указавать несколько столбцов
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [67]:
ratings.merge(movies, how = 'left', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [68]:
ratings.merge(movies, how = 'right', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [69]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

,date,source,visits
0,2019-11-01,organic,16825
1,2019-11-01,paid,1952
2,2019-11-02,organic,21890
3,2019-11-02,paid,376
4,2019-11-03,organic,19509


In [70]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

,date,orders
0,2019-11-01,198
1,2019-11-02,225
2,2019-11-03,201


In [80]:
visits_pivot = visits.pivot_table(index='date', columns='source', values='visits', fill_value=None)
visits_pivot

source,organic,paid
date,,
2019-11-01,16825.0,1952.0
2019-11-02,21890.0,376.0
2019-11-03,19509.0,NaN


In [74]:
visits_pivot.merge(orders, how = 'left', on = 'date')

,date,organic,paid,orders
0,2019-11-01,16825,1952,198
1,2019-11-02,21890,376,225
2,2019-11-03,19509,0,201


### Оптимизация хранения данных

In [81]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [82]:
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [83]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [84]:
len(ratings)

100004

In [85]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [87]:
len(movies)

9125

In [88]:
len(ratings) == len(joined)

True

In [89]:
joined.to_csv('joined_ratings.csv', index=False)

In [90]:
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [91]:
logs = joined[['userId', 'movieId', 'rating']].head()

In [92]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

9066

### Какой жанр имеет самые высокие рейтинги?

In [93]:
import numpy as np

In [94]:
genres = ['Drama', 'Action', 'Thriller']

In [95]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [96]:
len(ratings)

100004

In [97]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [98]:
len(movies)

9125

In [121]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [100]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

True

### Считаем рейтинг жанров

In [112]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [113]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [133]:
%%time
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

Wall time: 22.3 s


,userId,movieId,rating,timestamp,title,genres,Drama,Action,Thriller
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,2.5,NaN,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,3.0,NaN,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN,NaN,3.0
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,NaN,2.0,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,4.0,NaN,NaN


In [126]:
def genres_ratings_version_2(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    for genre in genres:
        if genre in row.genres:
            row[genre] = row.rating
    
    return rating

In [127]:
joined[genres] = joined.apply(genres_ratings_version_2, axis=1)
joined.head()

NameError: ("name 'rating' is not defined", 'occurred at index 0')

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

In [138]:
for genre in genres:
    print(genre, '-', round(joined[genre].mean(), 2))

Drama - 3.68
Action - 3.45
Thriller - 3.52


**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [159]:
# Вариант 1 - через датафрейм
ratings_movies = pd.read_csv('ml-latest-small/ratings.csv')
ratings_movies.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [157]:
ratings_filtered = ratings_movies.groupby(['userId']).agg({'timestamp': ['min', 'max'], 'movieId': 'count'}).reset_index()

ratings_filtered = ratings_filtered[ratings_filtered['movieId']['count'] > 100]

ratings_filtered['life_time'] = pd.to_datetime(ratings_filtered['timestamp']['max'], unit='s') \
- pd.to_datetime(ratings_filtered['timestamp']['min'], unit='s') # почему то не получается сделать переос строки методом обёртывания в скобки

ratings_filtered.head()

userId   timestamp             movieId          life_time
                  min         max   count                   
3       4   949778714   949982274     204    2 days 08:32:40
7       8  1154389340  1154474527     116    0 days 23:39:47
14     15   997937239  1469330735    1700 5455 days 22:38:16
16     17  1127468587  1127476640     363    0 days 02:14:13
18     19   855190091   855195373     423    0 days 01:28:02

In [217]:
# Черновик
import datetime
n = 3606404
print(datetime.timedelta(seconds = n))

41 days, 17:46:44


In [204]:
previous_user = None
user_lines = []
user_dict = {}

with open('ml-latest-small/ratings.csv') as f:
    f.readline()
    for line in f:
        line = line.strip().split(',')
        userId, movieId, rating, timestamp = line
#         print(userId, movieId, rating, timestamp)
        
        if previous_user != None:
            if previous_user != userId:
                if len(user_lines) > 100:
                    user_dict[previous_user] = max([int(x[3]) for x in user_lines]) - min([int(x[3]) for x in user_lines])
                previous_user = userId
                user_lines = [line]
            else:
                user_lines.append(line)   
        else:
            previous_user = userId
            user_lines.append(line)

In [219]:
user_df = pd.DataFrame(
    {
        'userId': list(user_dict.keys()),
        'life_time_sec': list(user_dict.values())
    }
)
user_df['life_time'] = pd.to_timedelta(user_df['life_time_sec'] * 10**9)
user_df.head()

,userId,life_time_sec,life_time
0,4,203560,2 days 08:32:40
1,8,85187,0 days 23:39:47
2,15,471393496,5455 days 22:38:16
3,17,8053,0 days 02:14:13
4,19,5282,0 days 01:28:02


In [226]:
# Вариант 2 - построчное считывание файла
previous_user = None
user_lines = []
user_dict = {}

with open('ml-latest-small/ratings.csv') as f:
    f.readline()
   
    for line in f:
        line = line.strip().split(',')
        userId, movieId, rating, timestamp = line

        if previous_user != None:
            if previous_user != userId:
                if len(user_lines) > 100:
                    sec = max([int(x[3]) for x in user_lines]) - min([int(x[3]) for x in user_lines])
                    user_dict[previous_user] = datetime.timedelta(seconds = sec)
                previous_user = userId
                user_lines = [line]
            else:
                user_lines.append(line)   
        else:
            previous_user = userId
            user_lines.append(line)

# 'user_dict' - словарь с id пользователя и временем его жизни в формате timedelta

# можно преобразовать словарь в датафрейм
user_df = pd.DataFrame(
    {
        'userId': list(user_dict.keys()),
        'life_time': list(user_dict.values())
    }
)
user_df.head()

,userId,life_time
0,4,2 days 08:32:40
1,8,0 days 23:39:47
2,15,5455 days 22:38:16
3,17,0 days 02:14:13
4,19,0 days 01:28:02


**Задание 2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [12]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [13]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [14]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [35]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [36]:
streams = [auto, air, rzd]
for stream in streams:
    client_base = client_base.merge(stream, how='outer', left_on='client_id', right_on='client_id')
print(client_base)

   client_id          address  auto_revenue  air_revenue  rzd_revenue
0        111  Комсомольская 4           NaN          NaN       1093.0
1        112   Энтузиастов 8а           NaN          NaN       2810.0
2        113  Левобережная 1а       57483.0          NaN      10283.0
3        114          Мира 14          83.0          NaN       5774.0
4        115        ЗЖБИиДК 1         912.0         81.0        981.0
5        116    Строителей 18        4834.0          4.0          NaN
6        117  Панфиловская 33          98.0         13.0          NaN
7        118     Мастеркова 4           NaN        173.0          NaN


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Можно совместить координаты магазинов и покупателей.
Из координат можно получить место нахождения и траекторию покупателя.